<a href="https://colab.research.google.com/github/Ashik9576/NLP/blob/main/Tekno_Assign.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Key Objectives**
Prepare the following deliverables and present under time constraints with the limited
information and with logical assumptions as required. Kindly study the details as mentioned
in the assignment. Kindly note any references and assumptions at the bottom.

# **Context and Data**
The dataset provided refers to the assessment of level of risk for specific areas and in the context of a construction site. The auditors need to proceed with different type of assessment replying to specific questions and judge the level of risk (At risk or Positive). A risk rating is associated to every risk based on the judgement of the auditors. Refer to the dataset provided to complete the requested key deliverables.

A brief explanation of the data is given here:

* Area Id (integer) - Unique id per area
* Area (string) - Area names
* Assessment (string) - Groups of Assessments
* Sub Assessments (string) - Groups of Sub Assessments
* Assessment Week (date) - Week in which the assessment was carried out
* Assessment Id (int) - Unique id per assessment
* Asset Id (int) - Unique id per asset
* Week range (string) - Week in which the assessment was carried out
* Questions (string) - Questions asked per assessment
* Risk or Positive (string) - Was the assessment deemed risky or not
* Risk rating (string) - Risk rating of the assessment
* Assessment date (date) - Exact date when the assessment was carried out
* Work Group Id (int) - Work Group Id of the assessment (One Work Group can have
many assessments)

# **Importing Required Libraries and mounting drive**

In [1]:
#importing Required Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from sklearn import preprocessing

In [2]:
#mounting drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# **Loading Dataset**

In [3]:
#loading dataset
df1=pd.read_excel('/content/drive/My Drive/ Teknobuilt Assignment/safety_ sheet.xlsx')

In [4]:
# creating copy of dataset
df=df1.copy()

# **File Structure and content**

In [5]:
#no. of row and column
print("We have {} rows.".format(df.shape[0]))
print("We have {} columns.".format(df.shape[1]))

We have 4572 rows.
We have 13 columns.


In [6]:
#accessing a rows
df.iloc[1,:]

Area Id                                                           572
Area                                          1000 - Process Train #1
Column1                                                           NaN
Assessment Week                                   2021-01-03 00:00:00
Assessment Id                                                     408
Assessment                                         Work Authorization
Asset Id                                                           37
Week Range                                      January 3 - January 9
Questions           Are all permit or work authorization documents...
Risk or Positive                                             Positive
Risk Rating                                                    Medium
Assessment Date                                   2021-01-05 00:00:00
Work Group Id                                                    1034
Name: 1, dtype: object

**At first galance, We have seen that the type of each variable and how they look like.**

# **MISSING VALUES**
Knowing about missing values is important because they indicate how much we dont's know about our data. Making inferences based on just a few cases is often unwise. In addition, many modelling procedures break down when missing values are involved and the corresponding rows will either have to be removed completely or the values need to be estimated somehow.

In [7]:
#check null values in Data frame
df.isnull().sum()

Area Id                0
Area                   0
Column1             4572
Assessment Week        0
Assessment Id          0
Assessment             0
Asset Id               0
Week Range             0
Questions              0
Risk or Positive       0
Risk Rating            0
Assessment Date        0
Work Group Id          0
dtype: int64

There is a column1 have missing value in each row so we have to drop this column

In [8]:
df.drop(df.columns[2], axis = 1, inplace = True)

In [9]:
#check null values in Data frame
df.isnull().sum()

Area Id             0
Area                0
Assessment Week     0
Assessment Id       0
Assessment          0
Asset Id            0
Week Range          0
Questions           0
Risk or Positive    0
Risk Rating         0
Assessment Date     0
Work Group Id       0
dtype: int64

# **Describing Dataset**

In [10]:
# cheking for first five rows
df.head()

,Area Id,Area,Assessment Week,Assessment Id,Assessment,Asset Id,Week Range,Questions,Risk or Positive,Risk Rating,Assessment Date,Work Group Id
0,572,1000 - Process Train #1,2021-01-03,404,General Safety,37,January 3 - January 9,Is barrier tape or rope being used?,At Risk,Medium,2021-01-07,1035
1,572,1000 - Process Train #1,2021-01-03,408,Work Authorization,37,January 3 - January 9,Are all permit or work authorization documents...,Positive,Medium,2021-01-05,1034
2,572,1000 - Process Train #1,2021-01-03,408,Work Authorization,37,January 3 - January 9,Are worker's qualified for job they are author...,Positive,Medium,2021-01-05,1034
3,572,1000 - Process Train #1,2021-01-03,408,Work Authorization,37,January 3 - January 9,Is the permit available at the worksite?,Positive,Medium,2021-01-05,1034
4,572,1000 - Process Train #1,2021-01-03,408,Work Authorization,37,January 3 - January 9,Is the work authorization documentation / perm...,Positive,Medium,2021-01-05,1034


In [11]:
# summery of dataframe
df.describe()

,Area Id,Assessment Id,Asset Id,Work Group Id
count,4572.000000,4572.000000,4572.0,4572.000000
mean,575.455599,403.274278,37.0,1034.539151
std,3.772808,4.109155,0.0,1.507075
min,572.000000,399.000000,37.0,1034.000000
25%,572.000000,399.000000,37.0,1034.000000
50%,576.000000,404.000000,37.0,1034.000000
75%,577.000000,406.000000,37.0,1035.000000
max,588.000000,415.000000,37.0,1061.000000


In [12]:
#information about dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4572 entries, 0 to 4571
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Area Id           4572 non-null   int64         
 1   Area              4572 non-null   object        
 2   Assessment Week   4572 non-null   datetime64[ns]
 3   Assessment Id     4572 non-null   int64         
 4   Assessment        4572 non-null   object        
 5   Asset Id          4572 non-null   int64         
 6   Week Range        4572 non-null   object        
 7   Questions         4572 non-null   object        
 8   Risk or Positive  4572 non-null   object        
 9   Risk Rating       4572 non-null   object        
 10  Assessment Date   4572 non-null   datetime64[ns]
 11  Work Group Id     4572 non-null   int64         
dtypes: datetime64[ns](2), int64(4), object(6)
memory usage: 428.8+ KB


# **Reformating Feature**
 Label Encoding on "Risk or Positive" and "Risk Rating" column

In [13]:
df['Risk or Positive'].value_counts()

Positive    4436
At Risk      136
Name: Risk or Positive, dtype: int64

In [14]:
df['Risk Rating'].value_counts()

Low       2231
Medium    2124
High       217
Name: Risk Rating, dtype: int64

In [15]:
 # label_encoder object knows how to understand word labels.
label_encoder = preprocessing.LabelEncoder()
 
# Encode labels in column 'Risk or Positive'.
df['Risk or Positive']= label_encoder.fit_transform(df['Risk or Positive'])

# Encode labels in column 'Risk Rating'.
df.loc[df["Risk Rating"] == "Low", "Risk Rating"] = 1
df.loc[df["Risk Rating"] == "Medium", "Risk Rating"] = 2
df.loc[df["Risk Rating"] == "High", "Risk Rating"] = 3


We encoded Risk = 0 and Positive = 1 in 'Risk or Positive' column.

We encoded Low = 1, Medium=2 and High = 3 in 'Risk Rating' column.

In [21]:
df.head(30)

,Area Id,Area,Assessment Week,Assessment Id,Assessment,Asset Id,Week Range,Questions,Risk or Positive,Risk Rating,Assessment Date,Work Group Id
0,572,1000 - Process Train #1,2021-01-03,404,General Safety,37,January 3 - January 9,Is barrier tape or rope being used?,0,2,2021-01-07,1035
1,572,1000 - Process Train #1,2021-01-03,408,Work Authorization,37,January 3 - January 9,Are all permit or work authorization documents...,1,2,2021-01-05,1034
2,572,1000 - Process Train #1,2021-01-03,408,Work Authorization,37,January 3 - January 9,Are worker's qualified for job they are author...,1,2,2021-01-05,1034
3,572,1000 - Process Train #1,2021-01-03,408,Work Authorization,37,January 3 - January 9,Is the permit available at the worksite?,1,2,2021-01-05,1034
4,572,1000 - Process Train #1,2021-01-03,408,Work Authorization,37,January 3 - January 9,Is the work authorization documentation / perm...,1,2,2021-01-05,1034
5,572,1000 - Process Train #1,2021-01-03,408,Work Authorization,37,January 3 - January 9,Is the work location consistent with the locat...,1,2,2021-01-05,1034
6,572,1000 - Process Train #1,2021-01-03,408,Work Authorization,37,January 3 - January 9,Were the correct permit (s) issued for the wor...,1,2,2021-01-05,1034
7,572,1000 - Process Train #1,2021-01-03,399,BBS,37,January 3 - January 9,Did the presenter allow workers to bring up an...,1,1,2021-01-05,1035
8,572,1000 - Process Train #1,2021-01-03,399,BBS,37,January 3 - January 9,Did the presenter attempt to get worker involv...,1,1,2021-01-05,1035
9,572,1000 - Process Train #1,2021-01-03,399,BBS,37,January 3 - January 9,Did the presenter try to related the tool box ...,1,1,2021-01-05,1035


In [17]:
df['Risk Rating'].value_counts()

1    2231
2    2124
3     217
Name: Risk Rating, dtype: int64

In [18]:
df['Week Range'].value_counts()

February 28 - March 6        1334
January 10 - January 16       699
February 21 - February 27     696
February 7 - February 13      613
January 31 - February 6       433
January 24 - January 30       377
January 17 - January 23       365
February 14 - February 20      40
January 3 - January 9          15
Name: Week Range, dtype: int64

In [19]:
df['Work Group Id'].value_counts()

1034    3318
1035     779
1038     258
1036     198
1037       8
1061       8
1040       3
Name: Work Group Id, dtype: int64

In [22]:
df.nunique()

Area Id              11
Area                 11
Assessment Week       9
Assessment Id        12
Assessment           12
Asset Id              1
Week Range            9
Questions           192
Risk or Positive      2
Risk Rating           3
Assessment Date      46
Work Group Id         7
dtype: int64

# **Exploratory Data Analysis**
**Area Id**